## Covid19 Vaccine Slots checking Code

In [16]:
import pandas as pd
import numpy as np
import requests as rq
from bs4 import BeautifulSoup as bs
import datetime
import json
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

##this above import is to minismise SSL Security warning in Macbook


'''
Scripts to get the Vaccination available slot for covid-19 centers with ease 

Open public API: https://apisetu.gov.in/public/marketplace/api/cowin
'''

base = datetime.datetime.today()
print(base)

numdays = 25
date_list = [base + datetime.timedelta(days=x) for x in range(numdays)]
print(date_list)
date_str = [x.strftime("%d-%m-%Y") for x in date_list]
date_str
age=58
print_flag = 'y'
print_flag = 'y'

headers={'User-Agent':"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36"}

def vaccine_state_data():
    '''function to print the state and its correponsding district names'''

    try:
        ## Checking statecode avaialble
        response_state = rq.get("https://cdn-api.co-vin.in/api/v2/admin/location/states",headers=headers,verify=False)
        json_data_state = json.loads(response_state.text)
        print()
        json_data_state['states']

        data_dist=[]
        for state_id in range(1,40):
            ##Checking till 40 state code as i checked there are 40 statecode availble!
#             try:
#                 print("For State_code:{} State_name: {} ".format(state_id,next(i['state_name'] for i in json_data_state['states'] if i['state_id']==state_id)))
#             except:
#                 print("For State_code:{} State_name: {}".format(state_id,'Not Available') )
#                 pass
            response_dist = rq.get("https://cdn-api.co-vin.in/api/v2/admin/location/districts/{}".format(state_id),headers=headers,verify=False)
            json_data = json.loads(response_dist.text)
            for _ in json_data["districts"]:
                
                # print(_["district_id"],'\t', _["district_name"])
                try:
                    data_dist.append([_["district_id"], _["district_name"],next(i['state_name'] for i in json_data_state['states'] if i['state_id']==state_id),state_id])
                except:
                    data_dist.append([_["district_id"], _["district_name"],None,state_id])
            # print("\n")

        df = pd.DataFrame(data_dist, columns = ['district_id', 'district_name','State_name','State_id'],index=None)
        df=df.sort_values(by=['State_id'], ascending=True)
        df_vacc_state = df.reindex(columns=['State_id','State_name','district_name','district_id'])
        print(len(df_vacc_state['district_id']))
        df_vacc_state['State_id'].nunique()
        return df_vacc_state


    except Exception as e:
        print("Exception ",e)



def vaccineSlotsByDist(age,district_name,state):
    try:
        available_df=[]
        age=age
        
        df_vacc_state=vaccine_state_data()
        try:
#             print(df_vacc_state)
            DIST_ID=list(df_vacc_state[(df_vacc_state['State_name']== state) & (df_vacc_state['district_name']==district_name)]['district_id'])[0]
            print("DIST_ID: ",DIST_ID)

        except:
            print("No Combintaiont found")
            return None
            pass

        for inp_date in date_str:
            URL = "https://cdn-api.co-vin.in/api/v2/appointment/sessions/public/calendarByDistrict?district_id={}&date={}".format(DIST_ID, inp_date)
            response = rq.get(URL,headers=headers,verify=False)
            if response.ok:
                resp_json = response.json()
                # print(json.dumps(resp_json, indent = 1))
                if resp_json["centers"]:
                    print("Available on: {}".format(inp_date))
                    if(print_flag=='y' or print_flag=='Y'):
                        for center in resp_json["centers"]:
                            for session in center["sessions"]:
                                if session["min_age_limit"] <= age :
#                                     print(session["min_age_limit"])
#                                     print("\t", center["name"])
#                                     print("\t", center["block_name"])
#                                     print("\t Price: ", center["fee_type"])
#                                     print("\t Available Capacity: ", session["available_capacity"])
                                    if(session["vaccine"] != ''):
#                                         print("\t Vaccine: ", session["vaccine"])
                                        pass
                                        
                                    else:
                                        
                                        session["vaccine"]='No Information'
#                                     print("\n\n")
                                    available_df.append([center["name"],center["block_name"],center["fee_type"],session["vaccine"],\
                                                        center['district_name'],center['state_name'],\
                                                        
                                                        center['pincode'],inp_date, session["available_capacity"], session["min_age_limit"]
                                                        
                                                        ])
                                                        
                           
                                 

                        
                else:
                    print("No available slots on {}".format(inp_date))





        # Create the pandas DataFrame for re grouping whole data into one particlur data frames
        info_vaccination = pd.DataFrame(available_df, columns = ['Vaccination Center', 'Block_name','Fee_type','Vaccine Type',\
            'District','State Name','pincode','Available Date','available capacity','Age Group'],index=None)
        
        indexNames = info_vaccination[ (info_vaccination['available capacity'] == 0) ].index
    
        info_vaccination.drop(indexNames , inplace=True)
       
        html=info_vaccination.set_index(['Available Date', 'State Name','District','pincode'])
        
        

#         html= html.to_html()#classes='table table-hover ')
#         html_ind_corona = html.replace('<thead>','<thead class="thead-dark">')
        return html#_ind_corona

    except Exception as e:
        print("Exception ",e)

    
    
    

2021-05-13 19:49:03.779444
[datetime.datetime(2021, 5, 13, 19, 49, 3, 779444), datetime.datetime(2021, 5, 14, 19, 49, 3, 779444), datetime.datetime(2021, 5, 15, 19, 49, 3, 779444), datetime.datetime(2021, 5, 16, 19, 49, 3, 779444), datetime.datetime(2021, 5, 17, 19, 49, 3, 779444), datetime.datetime(2021, 5, 18, 19, 49, 3, 779444), datetime.datetime(2021, 5, 19, 19, 49, 3, 779444), datetime.datetime(2021, 5, 20, 19, 49, 3, 779444), datetime.datetime(2021, 5, 21, 19, 49, 3, 779444), datetime.datetime(2021, 5, 22, 19, 49, 3, 779444), datetime.datetime(2021, 5, 23, 19, 49, 3, 779444), datetime.datetime(2021, 5, 24, 19, 49, 3, 779444), datetime.datetime(2021, 5, 25, 19, 49, 3, 779444), datetime.datetime(2021, 5, 26, 19, 49, 3, 779444), datetime.datetime(2021, 5, 27, 19, 49, 3, 779444), datetime.datetime(2021, 5, 28, 19, 49, 3, 779444), datetime.datetime(2021, 5, 29, 19, 49, 3, 779444), datetime.datetime(2021, 5, 30, 19, 49, 3, 779444), datetime.datetime(2021, 5, 31, 19, 49, 3, 779444), dat

In [18]:
val=vaccineSlotsByDist(age=32,district_name='Hyderabad',state='Telangana')
val

data=['Central Delhi','East Delhi','North Delhi','Shahdara','West Delhi','New Delhi']
dat1=[141,145,140,146,147,143]

t1=['Odisha']

for i in dat1:
    print(i)
    fa=vaccineSlotsByDist(32,i)
    da=vaccineSlotsByDist(55,i)
    print(fa,da)

In [9]:
print(val)

None


In [19]:
val=vaccineSlotsByDist(age=55,district_name='Jajpur',state='Odisha')
val


761
DIST_ID:  460
Available on: 13-05-2021
Available on: 14-05-2021
Available on: 15-05-2021
Available on: 16-05-2021
Available on: 17-05-2021
Available on: 18-05-2021
Available on: 19-05-2021
Available on: 20-05-2021
Available on: 21-05-2021
Available on: 22-05-2021
Available on: 23-05-2021
Available on: 24-05-2021
Available on: 25-05-2021
Available on: 26-05-2021
Available on: 27-05-2021
Available on: 28-05-2021
No available slots on 29-05-2021
No available slots on 30-05-2021
No available slots on 31-05-2021
No available slots on 01-06-2021
No available slots on 02-06-2021
No available slots on 03-06-2021
No available slots on 04-06-2021
No available slots on 05-06-2021
No available slots on 06-06-2021


,,,,Vaccination Center,Block_name,Fee_type,Vaccine Type,available capacity,Age Group
Available Date,State Name,District,pincode,,,,,,


## Sending Available slots

In [5]:
import pandas as pd
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
from smtplib import SMTP
import smtplib
import sys


# df_1 = ([1,2,3,5])
# df_2 = ([10,20,30,50])
# df_test =pd.concat([pd.DataFrame(df_1),pd.DataFrame(df_2)],axis=1)
df_test=val

recipients = ['rcpient@gmail.com'] 
emaillist = [elem.strip().split(',') for elem in recipients]
msg = MIMEMultipart()
msg['Subject'] = "dataframe table TEST FROM GMAIL"
msg['From'] = 'sender@gmail.com'

html = """\
        <html>
          <head></head>
          <body>
            {0}
          </body>
        </html>
""".format(df_test.to_html())

part1 = MIMEText(html, 'html')
msg.attach(part1)

try:
    """Checking for connection errors"""
    print("Sending...")

    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.ehlo()#NOT NECESSARY
    server.starttls()
    server.ehlo()#NOT NECESSARY
    server.login('sender@gmail.com','pswd')
    server.sendmail(msg['From'], emaillist , msg.as_string())
    server.close()
    print("Mail Sent Successfully")

except Exception as e:
    print("Error for connection: {}".format(e))

Sending...
Mail Sent Successfully


## Using Twitter API to send the data

In [1]:
import tweepy
import webbrowser
import time
from tweepy import OAuthHandler

import tweepy
from tweepy import StreamListener
from tweepy import Stream

In [2]:
CONSUMER_KEY=''
CONSUMER_SECRET=''
Access_Token='3056269028-'
Access_Token_Secret='

In [3]:
# callback_uri='oob'
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
# auth.set_access_token("ACCESS_TOKEN", "ACCESS_TOKEN_SECRET")
# redirect_url = auth.get_authorization_url()
# print(redirect_url)
auth

In [4]:
auth.set_access_token(Access_Token,Access_Token_Secret)

In [5]:
# Create API object
api = tweepy.API(auth, wait_on_rate_limit=True,
    wait_on_rate_limit_notify=True)
# api.verify_credentials()


In [17]:
dir(api)
# me=api.me()

timeline = api.home_timeline(verify=False)
for tweet in timeline:
    print(f"{tweet.user.name} said {tweet.text}")

In [13]:
tweets_listener = StreamListener(api)
twitterStream = tweepy.Stream(auth,tweets_listener, verify = False)
twitterStream

In [72]:
api = tweepy.API(auth)

try:
    api.verify_credentials()
    print("Authentication OK")
except:
    print("Error during authentication")

Error during authentication
